![Hypernets](hypernets/resources/img/logo.png)



# Field Deployment  : step by step

## &#x2022;  Step 1 : Yoctopuce
### Make sure you are able to retrieve meteo and GPS signal data 

In [ ]:
from ipywidgets import HBox, VBox, Button, Label
from IPython.display import display

gps_show = Button(description="Get GPS location")
meteo = Button(description="Get Meteo Data")

gps_lbl = Label(value="")
meteo_lbl = Label(value="")


@gps_show.on_click
def show_gps_coords(_):
    from hypernets.scripts.yocto_gps import get_gps
    gps_lbl.value = get_gps(return_float=False)
    
@meteo.on_click
def show_meteo(_):
    from hypernets.scripts.yocto_meteo import get_meteo
    v_meteo = "   ".join([str(val) + unit for val, unit in get_meteo()])
    meteo_lbl.value = v_meteo
    
display(VBox((HBox((gps_show, gps_lbl)), HBox((meteo, meteo_lbl)))))

Here, you should double check if the [GPS] latitude and longitude are good in your configuration file :
http://10.42.0.1:8888/edit/config_hypernets.ini


### &#x2022;  Yocto Relays

In [ ]:
from hypernets.scripts.relay_command import set_state_relay
from ipywidgets import HBox, FloatText, Button
from IPython.display import display

relay_on =  [Button(description=f"{i} ON") for i in range(1, 7, 1)]
relay_off = [Button(description=f"{i} OFF") for i in range(1, 7, 1)]

def click_on(b):
    id_relay = int(b.description[0])
    set_state_relay(id_relay, "on")
    
def click_off(b):
    id_relay = int(b.description[0])
    set_state_relay(id_relay, "off")

for relay in relay_on:
    relay.on_click(click_on)

for relay in relay_off:
    relay.on_click(click_off)
    
display(VBox((HBox((relay_on)), HBox((relay_off)))))

## &#x2022;  Step 2 : Mechanical position
### a) Use a bubble level on the mast to ensure the verticality

![pantilt_level](hypernets/resources/img/pan_tilt_bubble.png)


### b) Move the instrument into a horizontal position and adjust tilt if necessary (instrument must be horizontal when 90 or -90 asked position)

Edit config : http://10.42.0.1:8888/edit/config_hypernets.ini

In [ ]:
from hypernets.scripts.pan_tilt import move_to
from ipywidgets import HBox, FloatText, Button
from IPython.display import display

pan = FloatText(description="Pan :")
tilt = FloatText(description="Tilt :")
move = Button(description="Move Pan-Tilt")


@move.on_click
def move_pan_tilt(_):
    move_to(None, pan.value, tilt.value, verbose=False, wait=False)

display(HBox(((move, pan, tilt))))

## &#x2022; Step 3 : Check pointing

### Solar Position Algorithm (SPA)
Here are two differents ways to compute the sun position : 
1) According to the GPS values <br/>
2) According to utctime from the machine and latitude / longitude of the config_hypernets.ini file.


In [ ]:
from hypernets.scripts.spa.spa_hypernets import spa_from_gps, spa_from_datetime

print("="*80 + "\nSPA from GPS :")
try:
    print(spa_from_gps())
    
except ValueError as e:
    print(f"Error : {e}")
print("="*80 + "\nSPA from datetime :")
print(spa_from_datetime(verbose=True))

### Point to the sun

In [ ]:
from hypernets.scripts.pan_tilt import move_to
from hypernets.scripts.spa.spa_hypernets import spa_from_datetime, spa_from_gps
from ipywidgets import HBox, Button, Label
from IPython.display import display

point_gps = Button(description="(gps)")
point_datetime = Button(description="config lat/long")

sun_gps_lbl = Label(value="")
sun_datetime_lbl = Label(value="")

@point_datetime.on_click
def point_sun_datetime(_):
    azimuth_sun, zenith_sun = spa_from_datetime()
    sun_datetime_lbl.value = f"Sun position : {azimuth_sun}, {zenith_sun}"
    move_to(None, azimuth_sun, 180-zenith_sun, verbose=False, wait=False)

@point_gps.on_click
def point_sun_gps(_):
    azimuth_sun, zenith_sun = spa_from_gps()
    sun_gps_lbl.value = f"Sun position : {azimuth_sun}, {zenith_sun}"
    move_to(None, azimuth_sun, 180-zenith_sun, verbose=False, wait=False)    
    
display(VBox((HBox((point_gps, sun_gps_lbl)), HBox((point_datetime, sun_datetime_lbl)))))
#display((HBox((point_datetime, sun_datetime_lbl))))

### Take a Picture 
Turn the instrument on thanks and try take a picture :

In [ ]:
from hypernets.scripts.call_radiometer import take_picture
from IPython.display import Image
from ipywidgets import HBox, Button, Label
from IPython.display import display

take_pic = Button(description="Take a picture")

output_lbl = Label(value="")

@take_pic.on_click
def take_picture_action(_):
    output_lbl.value = "Wait..."
    try:
        take_picture(None)
        output_lbl.value = "Ok (see http://10.42.0.1:8888/tree/DATA)"
        #display(pic_sun)
        # stream = take_picture(return_stream=True)
        # pic_sun = Image(stream)        
    except Exception as e:
        output_lbl.value = f"Error : {e}"
        

display(HBox((take_pic, output_lbl)))

## &#x2022; Take Spectrum (in progress...)

In [ ]:
from ipywidgets import Combobox, ToggleButtons, Dropdown, VBox, HBox

radiometer = Dropdown(description='Radiometer', options=['VNIR', 'SWIR', 'BOTH'])
entrance = Dropdown(description='Entrance', options=['Radiance', 'Irradiance', 'Dark'])
IT_Vnir = Dropdown(description='IT_Vnir', options=tuple([0]+[pow(2, i) for i in range(16)]))
IT_Swir = Dropdown(description='IT_Swir', options=tuple([0]+[pow(2, i) for i in range(16)]))

out = HBox([VBox([radiometer, entrance]), VBox([IT_Vnir, IT_Swir])])
display(out)

In [ ]:
from hypernets.virtual.create_metadata import parse_config_metadata
print("=" * 31 + " METADATA PREVIEW " + "=" * 31)
print(parse_config_metadata())
print("=" * 80)